In [1]:
import cv2
import os
from tqdm import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import random
import urllib
import requests
import webbrowser
import shutil
import collections
import re
import time
import json
from glob import glob
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
import tensorflow.keras.layers as layers
from tensorflow.keras.initializers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import *

from sklearn.utils import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *

import keras.backend as K
from scipy.stats import *
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
import xgboost as xgb

print('Using Tensorflow version:', tf.__version__)

Using Tensorflow version: 2.3.1


In [2]:
imgSize = 28
BS = 8
EPOCHS = 10
INIT_LR = 3e-4
opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [3]:
trainAug = ImageDataGenerator(rotation_range=5, rescale=1./255, validation_split=0.2)
validAug = ImageDataGenerator(rescale=1./255, validation_split=0.3)

In [4]:
train_generator = trainAug.flow_from_directory('../input/mnistasjpg/trainingSet/trainingSet/',
                                               target_size=(imgSize, imgSize),
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               batch_size=BS, 
                                               shuffle=False,
                                               subset='training')
valid_generator = validAug.flow_from_directory('../input/mnistasjpg/trainingSet/trainingSet/',
                                               target_size=(imgSize, imgSize),
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               batch_size=BS, 
                                               shuffle=False,
                                               subset='validation')

Found 33604 images belonging to 10 classes.
Found 12596 images belonging to 10 classes.


In [5]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(10, activation = "softmax"))

In [6]:
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model = self.model

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model = self.model

In [7]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
save_best_model = SaveBestModel()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

In [8]:
H = model.fit(train_generator,
              steps_per_epoch=len(train_generator) // BS,
              validation_data=valid_generator,
              validation_steps=len(valid_generator) // BS,
              callbacks=[save_best_model],
              epochs=EPOCHS)

Epoch 1/10
525/525 [==============================] - 43s 82ms/step - loss: 0.9524 - accuracy: 0.6857 - val_loss: 0.1091 - val_accuracy: 0.9681
Epoch 2/10
525/525 [==============================] - 33s 63ms/step - loss: 0.1727 - accuracy: 0.9483 - val_loss: 0.6102 - val_accuracy: 0.8214
Epoch 3/10
525/525 [==============================] - 32s 60ms/step - loss: 0.1277 - accuracy: 0.9657 - val_loss: 0.0350 - val_accuracy: 0.9904
Epoch 4/10
525/525 [==============================] - 31s 59ms/step - loss: 0.1031 - accuracy: 0.9705 - val_loss: 0.0438 - val_accuracy: 0.9879
Epoch 5/10
525/525 [==============================] - 30s 57ms/step - loss: 0.0937 - accuracy: 0.9745 - val_loss: 0.0217 - val_accuracy: 0.9936
Epoch 6/10
525/525 [==============================] - 29s 55ms/step - loss: 0.0868 - accuracy: 0.9800 - val_loss: 0.0183 - val_accuracy: 0.9943
Epoch 7/10
525/525 [==============================] - 28s 54ms/step - loss: 0.0876 - accuracy: 0.9767 - val_loss: 0.0178 - val_accuracy:

In [9]:
best_model = save_best_model.best_model

In [10]:
tf.saved_model.save(best_model, 'my_model')

In [11]:
converter = tf.lite.TFLiteConverter.from_saved_model('my_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)